### 配置文件验证

In [ ]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config.yaml') # 读取配置文件

In [5]:
from modelModule.model5 import VAE5
import torch
model = VAE5(dim=16, pro_types=pro_types)

### 得到数据集相关属性

In [1]:
import pickle
import pandas as pd
data_pd = pd.read_csv('Letter_train.csv')
col_names = list(data_pd)
# df[col_name].value_counts()
pro_types = [('discrete',16) for _ in col_names]
# with open('Letter_pro_type.pkl', 'wb') as f:
#     pickle.dump(pro_types, f)

### dataloader验证

In [ ]:
import torch
from dataModule.dataset1 import FlatDataset
from torch.utils.data import DataLoader

In [ ]:
dataset = FlatDataset('Credit_Card无标签.csv')
print(dataset.Max_Val.shape)
print(dataset.Min_Val)

In [ ]:
dataset = FlatDataset('Letter_train.csv')
train_dataloader = DataLoader(dataset, 
                             batch_size=10, 
                             collate_fn=dataset.collater, 
                             shuffle=True,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [ ]:
test_data = imputed_data * global_max + global_min # 广播效应

In [ ]:
a = torch.randint(0,10, size=(5,4,3)).float()
print(a)
f = torch.nn.Softmax(dim=1)
f(a)


In [ ]:
a = torch.randint(0,10,size=(5,12,4)).float()
print(a)
model = torch.nn.Linear(4,10)
model(a[:,1,:]).shape

In [ ]:
a[:,0].reshape(-1,1)

In [ ]:
model(a[:,0].reshape(-1,1,1)).permute(0, 2, 1).shape

In [ ]:
isinstance(model, torch.nn.Embedding)

In [ ]:
a = torch.randint(0,2,size=(5, 3))
b = torch.nn.Parameter(torch.randn(1, 4))
print(b)
print(a)

miss_matric = a.unsqueeze (-1).expand(5,3,4)
print(miss_matric)
miss_matric*b

In [ ]:
a = torch.randint(0,10,size=(5,3)).float()
model = torch.nn.Conv1d(in_channels=1, out_channels=8, kernel_size=1, stride=1)
print(a)
embedding_out = torch.tensor([])
embedding_out = torch.cat((embedding_out, model(a[:,0].reshape(-1,1,1)).permute(0, 2, 1)), dim = 1)
embedding_out = torch.cat((embedding_out, model(a[:,1].reshape(-1,1,1)).permute(0, 2, 1)), dim = 1)
print(embedding_out.shape)
embedding_out

In [ ]:
b.shape

### 模型精度测定

In [ ]:
from dataModule.dataset1 import FlatDataset
# from modelModule.model1 import VAE
# from modelModule.model2 import VAE2
from modelModule.model4 import VAE4
from torch.utils.data import DataLoader
from utils import result_show, get_missing
import torch
import numpy as np

In [ ]:
## 模型参数载入
model = VAE4(dim=23) # nhead=3
checkpoint_path = r'模型参数保存/Credit_model4_norm_way_mean_norm_/version_1/checkpoints/epoch=38-step=936.ckpt'    
data_norm = 'mean_norm'  # 数据集采用正则化方式minmax_norm,mean_norm
train_csv = r'Credit_Card_test.csv'
test_csv = r'Credit_Card_test.csv'

checkpoint = torch.load(checkpoint_path)
model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
model.eval()
## 载入训练集的均值(最小)方差(最大)
train_dataset = FlatDataset(train_csv, data_norm=data_norm)
model.get_global_min_max(global_max=train_dataset.Max_Val, global_min=train_dataset.Min_Val)
print('载入成功')

In [ ]:
test_dataset = FlatDataset(test_csv, data_norm=data_norm)
data_size = 10
src_data = np.stack([test_dataset[index]['src_data'] for index in range(data_size)], axis=0)
normal_data = np.stack([test_dataset[index]['global_normal_data'] for index in range(data_size)], axis=0)
_, miss_matrix = get_missing(src_data, 0.3) # 得到缺失矩阵
input = src_data.copy()
input[miss_matrix == 0] = np.nan

In [ ]:
imputed_data, output = model.inference(input, miss_matrix) # inference输出的结果已经重构后的结果
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()

In [ ]:
for i in range(23):
    print(src_data[0][i], '————', imputed_data[0][i])

In [ ]:
imputed_data[2]

In [ ]:
imputed_data[1]

In [ ]:
result_show(src_data, imputed_data, miss_matrix)

In [ ]:
imputed_data

In [ ]:
imputed_data.shape

In [ ]:
# test_dataset = FlatDataset('Letter_test.csv', data_norm='mean_norm') # minmax_norm
# index = 102
# src_data, M_matrix, miss_data = test_dataset[index]['src_data'].reshape(1,-1), test_dataset[index]['global_miss_matrix'].reshape(1,-1), test_dataset[index]['global_miss_data'].reshape(1,-1)
# src_data = test_dataset[index]['src_data'].reshape(1,-1)
# normal_data = test_dataset[index]['global_normal_data'].reshape(1,-1)
# miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

# imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
# res_imputed_data = np.around(restore_data(imputed_data, test_dataset.Max_Val, test_dataset.Min_Val))
# result_show(src_data, res_imputed_data, M_matrix)

In [ ]:
# batch测试
# src_data, miss_data, M_matrix = batch['src_data'], batch['miss_data'], batch['miss_matrix']
# normal_data = batch['normal_data']
# imputed_data, _, _ = model(miss_data, M_matrix)
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()
# res_imputed_data = np.around(restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val))